<a href="https://colab.research.google.com/github/DmitriyBRM/practice-aia-2024/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_0_8_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
import io
from datetime import *
from PIL import Image
from IPython.display import display

current_date = datetime.today().date()
current_date = f'{current_date.year}-{current_date.month}-{current_date.day}'
url_APOD = 'https://api.nasa.gov/planetary/apod?api_key=bShS9hPraflofTnuICjXyW9SREIhGPPoEK5j3k6z'
move = None

def send_request(url):

    response = requests.get(url)

    print(response.status_code)

    if response.status_code == 200:
        return response.json()


def status_code(url):

    response = requests.get(url)

    return response.status_code

def apod(url_APOD):

    info = send_request(url_APOD)
    image_url = info['url']
    res = requests.get(image_url)
    explanation = info['explanation']
    img = Image.open(io.BytesIO(res.content))

    display(img)
    print(explanation)


def marsRover_photo(url_marsRover):
    status = status_code(url_marsRover)
    if status == 200:
        info = send_request(url_marsRover)

        for photo in info['photos']:
            url_photo = photo['img_src']
            res = requests.get(url_photo)
            img = Image.open(io.BytesIO(res.content))

            display(img)
        else:
            print('фото нет')

    else:
        print('фото нет')



def SBDB(url_SBDB):
    if status_code(url_SBDB) == 200:

        info = send_request(url_SBDB)
        k = 0

        for date, names  in info['near_earth_objects'].items():
            print(f'дата({date})')

            for info in names:
                k+=1

                if info['is_potentially_hazardous_asteroid'] == True:
                    danger = 'есть'

                else:
                    danger = 'отсутствует'
                print(f'{k}) название: {info["name"]}, размер {info["estimated_diameter"]["meters"]["estimated_diameter_max"]}, опасность {danger}')


    else:
        print('ошибка')


def space_weather(url_weather_FLR, url_weather_GST):

    infoFLR = send_request(url_weather_FLR)
    infoGST = send_request(url_weather_GST)

    print('информация о солнечных вспышках:')

    if infoFLR != []:
        for data in infoFLR:
            print(f'id:  {data["flrID"]}')

            if data['note'] == '':
                print('описания нет')

            else:
                print(f'описание:  {data["note"]}')
    else:
        print('нет иформации')

    print('информация о геомагнитной буре:')

    for data in infoGST:
        print(f'время наблюдения: {data["allKpIndex"][0]["observedTime"]}')
        print(f'глобальный показатель авроральной активности: {data["allKpIndex"][0]["kpIndex"]}')

    else:
        print('нет информации')


def main(url):
    data = send_request(url)
    message = (data)

    print(message)



while move != 4:
    print('выберете функию:')
    print('1) Астрономическая картинка дня (APOD)')
    print('2) Фотографии с марсохода')
    print('3) Объекты, сближающиеся с Землей (ОСЗ)')
    print('4) Данные о космической погоде')
    print('5) стоп')
    move = input()

    if move == '1':
        apod(url_APOD)

    elif move == '2':
        cameraType = {1:'FHAZ', 2:'RHAZ', 3:'MAST', 4:'CHEMCAM', 5:'MAHLI', 6:'MARDI', 7:'NAVCAM'}
        sol = input('введите сол: ')

        print(cameraType)

        camType = int(input('введите тип камеры: '))

        ctype = cameraType[camType]
        url_marsRover = f'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?sol={sol}&api_key=bShS9hPraflofTnuICjXyW9SREIhGPPoEK5j3k6z&camera={ctype}'

        marsRover_photo(url_marsRover)

    elif move == '3':
        chooseDate = input('введите дату в формате гггг-мм-дд: ')
        url_SBDB = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={chooseDate}&end_date={chooseDate}&api_key=bShS9hPraflofTnuICjXyW9SREIhGPPoEK5j3k6z'
        SBDB(url_SBDB)

    elif move == '4':
        chooseDate = input('введите дату в формате гггг-мм-дд: ')
        url_weather_FLR = f'https://api.nasa.gov/DONKI/FLR?startDate={chooseDate}&endDate={chooseDate}&api_key=bShS9hPraflofTnuICjXyW9SREIhGPPoEK5j3k6z'
        url_weather_GST = f'https://api.nasa.gov/DONKI/GST?startDate={chooseDate}&endDate={chooseDate}&api_key=bShS9hPraflofTnuICjXyW9SREIhGPPoEK5j3k6z'

        space_weather(url_weather_FLR, url_weather_GST)

    elif move == '5':
        break






выберете функию:
1) Астрономическая картинка дня (APOD)
2) Фотографии с марсохода
3) Объекты, сближающиеся с Землей (ОСЗ)
4) Данные о космической погоде
5) стоп
5


# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests

url = 'https://api.artic.edu/api/v1/artworks'

def send_request(page):

    response = requests.get(f'{url}?page={page}')
    if response.status_code == 200:
        data = response.json()
        return data['data'], data['pagination']


def artistName(arts, name):

    for art in arts:
        if name in art['artist_display']:
            f = [art]

    return f


def art(art):
    title = art.get('title', 'Без названия')
    artist = art.get('artist_display', 'Неизвестный')
    date = art.get('date_display', 'Неизвестная дата')
    medium = art.get('medium_display', 'Неизвестно')

    print(f"Название: {title}, художник: {artist} (Дата: {date}), Носитель: {medium}")


move = None

page = 1
while move != 4:
    artworks, pages = send_request(page)
    print(f"Страница {page} из {pages.get('total_pages', 1)}")

    k = 0
    for artwork in artworks:
        k+= 1
        print(f'{k}) {artwork.get("title")}, ({artwork.get("artist_title")})')

    print("1) Фильтрация по исполнителю")
    print("2) Следующая страница")
    print("3) Предыдущая страница")
    print("4) Выход")

    move = input("Введите номер команды: ")

    if move == '1':
        name = input("Введите имя художника: ")
        filtered_artworks = artistName(artworks, name)
        k+= 1

        for artwork in filtered_artworks:
            k+= 1
            print(f'{k}) {artwork.get("title")}')

        a = int(input("Введите номер произведения для просмотра деталей: ")) - 1
        art(filtered_artworks[a])

    elif move == '2':
        page += 1

    elif move == '3':
        page -= 1

    elif move == '4':
        break

Страница 1 из 10528
1) The Philosopher's Conquest, (Giorgio de Chirico)
2) The Superb Lily, from The Temple of Flora, (Richard Earlom)
3) The Superb Lily, from The Temple of Flora, (Richard Earlom)
4) The Annunciation, (Jean Hey, (the Master of Moulins))
5) Henri Degas and His Niece Lucie Degas (The Artist's Uncle and Cousin), (Hilaire Germain Edgar Degas)
6) Alexander Grant, (Cosmo Alexander)
7) City Landscape, (Joan Mitchell)
8) Self-Portrait, (Max Beckmann)
9) Champs de Mars: The Red Tower, (Robert Delaunay)
10) Portrait of Marevna, (Diego Rivera)
11) Interior at Nice, (Henri Matisse)
12) Untitled (Brown and Gray), (Mark Rothko)
1) Фильтрация по исполнителю
2) Следующая страница
3) Предыдущая страница
4) Выход
Введите номер команды: 1
Введите имя художника: Max Beckmann
14) Self-Portrait
Введите номер произведения для просмотра деталей: 1
Название: Self-Portrait, художник: Max Beckmann
German, 1884–1950 (Дата: 1937), Носитель: Oil on canvas
Страница 1 из 10528
1) The Philosopher's C

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
import requests
import time
from datetime import datetime


def current_price(crypto_id):
    url = f'https://api.coingecko.com/api/v3/simple/price?ids={crypto_id}&vs_currencies=usd&include_24hr_change=true'
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()[crypto_id]

    else:
        print("Ошибка получения данных")
        return None


def add(portfolio, crypto_id, amount):
    if crypto_id in portfolio:
        portfolio[crypto_id] += amount

    else:
        portfolio[crypto_id] = amount


def remove(portfolio, crypto_id, amount):
    if crypto_id in portfolio:
        portfolio[crypto_id] -= amount

        if portfolio[crypto_id] <= 0:
            del portfolio[crypto_id]


def get_total_value(portfolio):
    total_value = 0

    for crypto_id, amount in portfolio.items():
        price_data = current_price(crypto_id)

        if price_data:
            total_value += price_data['usd'] * amount
    return total_value


def display_changes(portfolio):
    for crypto_id in portfolio.keys():
        price_data = current_price(crypto_id)

        if price_data:
            print(f"{crypto_id.upper()}: {price_data['usd']} USD, "
                  f"24h Change: {price_data['usd_24h_change']:.2f}%")


def historical_price_data(crypto_id):
    end_time = int(time.time())
    start_time = end_time - 24 * 60 * 60
    url = f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range?vs_currency=usd&from={start_time}&to={end_time}'
    response = requests.get(url)
    historical_data = response.json()
    return historical_data['prices']


def display_historical_data(crypto_id):
        prices = historical_price_data(crypto_id)
        print(f'Исторические данные для {crypto_id}:')

        for price in prices:
            time = datetime.fromtimestamp(price[0] / 1000)
            value = price[1]
            print(f'{time.strftime("%Y-%m-%d %H:%M:%S")}: {value:.2f} USD')


portfolio = {}

add(portfolio, 'bitcoin', 0.05)
add(portfolio, 'ethereum', 1.2)

total_value = get_total_value(portfolio)
print(f"Общая стоимость портфеля: {total_value:.2f} USD")

display_changes(portfolio)


remove(portfolio, 'bitcoin', 0.01)
total_value = get_total_value(portfolio)
print(f"Общая стоимость портфеля после удаления: {total_value:.2f} USD")

display_historical_data('bitcoin')

# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код